# Download Blast+ (contains psi-blast) to compute PSSM

In [15]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.16.0+-x64-linux.tar.gz

--2025-03-05 17:38:25--  https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.16.0+-x64-linux.tar.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257516053 (246M) [application/x-gzip]
Saving to: ‘ncbi-blast-2.16.0+-x64-linux.tar.gz’

ncbi-blast-2.16.0+- 100%[===================>] 245.59M  12.3MB/s    in 35s     

2025-03-05 17:39:00 (7.09 MB/s) - ‘ncbi-blast-2.16.0+-x64-linux.tar.gz’ saved [257516053/257516053]



In [16]:
!tar -zxvf ncbi-blast-2.16.0+-x64-linux.tar.gz
!rm ncbi-blast-2.16.0+-x64-linux.tar.gz

ncbi-blast-2.16.0+/
ncbi-blast-2.16.0+/ChangeLog
ncbi-blast-2.16.0+/bin/
ncbi-blast-2.16.0+/bin/rpsblast
ncbi-blast-2.16.0+/bin/blastn_vdb
ncbi-blast-2.16.0+/bin/blast_vdb_cmd
ncbi-blast-2.16.0+/bin/makeprofiledb
ncbi-blast-2.16.0+/bin/blastp
ncbi-blast-2.16.0+/bin/deltablast
ncbi-blast-2.16.0+/bin/blastx
ncbi-blast-2.16.0+/bin/rpstblastn
ncbi-blast-2.16.0+/bin/psiblast
ncbi-blast-2.16.0+/bin/tblastx
ncbi-blast-2.16.0+/bin/tblastn_vdb
ncbi-blast-2.16.0+/bin/blast_formatter_vdb
ncbi-blast-2.16.0+/bin/segmasker
ncbi-blast-2.16.0+/bin/convert2blastmask
ncbi-blast-2.16.0+/bin/blastdbcmd
ncbi-blast-2.16.0+/bin/get_species_taxids.sh
ncbi-blast-2.16.0+/bin/dustmasker
ncbi-blast-2.16.0+/bin/cleanup-blastdb-volumes.py
ncbi-blast-2.16.0+/bin/blastn
ncbi-blast-2.16.0+/bin/windowmasker
ncbi-blast-2.16.0+/bin/makeblastdb
ncbi-blast-2.16.0+/bin/blastdb_aliastool
ncbi-blast-2.16.0+/bin/update_blastdb.pl
ncbi-blast-2.16.0+/bin/makembindex
ncbi-blast-2.16.0+/bin/blast_formatter
ncbi-blast-2.16.0+/bin/l

## Download Seqtk to fetch sequences from diamond database

In [ ]:
!git clone https://github.com/lh3/seqtk.git
%cd seqtk
!make
!cd ..

## Filtered Diamond Output with E-value $1 \times 10^{-5}$

In [1]:
!awk '$11 < 1e-5 {print $2}' ../../dataset/diamond_results/test_diamond_output.m8 | sort -u > ../../dataset/pssm/test_subject_ids.txt

In [2]:
!awk '$11 < 1e-5 {print $2}' ../../dataset/diamond_results/valid_diamond_output.m8 | sort -u > ../../dataset/pssm/valid_subject_ids.txt

In [3]:
!awk '$11 < 1e-5 {print $2}' ../../dataset/diamond_results/train_diamond_output.m8 | sort -u > ../../dataset/pssm/train_subject_ids.txt

In [ ]:
!seqtk/seqtk subseq ../../dataset/dimond_db/uniref90.fasta ../../dataset/pssm/test_subject_ids.txt > ../../dataset/pssm/test_filtered_seqs.fasta

In [ ]:
!seqtk/seqtk subseq ../../dataset/dimond_db/uniref90.fasta ../../dataset/pssm/valid_subject_ids.txt > ../../dataset/pssm/valid_filtered_seqs.fasta

In [ ]:
!seqtk/seqtk subseq ../../dataset/dimond_db/uniref90.fasta ../../dataset/pssm/train_subject_ids.txt > ../../dataset/pssm/train_filtered_seqs.fasta

## Make diamond output a Blast Database

In [ ]:
!ncbi-blast-2.16.0+/bin/makeblastdb -in ../../dataset/pssm/test_filtered_seqs.fasta -dbtype prot -out ../../dataset/pssm/test_db

In [ ]:
!ncbi-blast-2.16.0+/bin/makeblastdb -in ../../dataset/pssm/valid_filtered_seqs.fasta -dbtype prot -out ../../dataset/pssm/valid_db

In [ ]:
!ncbi-blast-2.16.0+/bin/makeblastdb -in ../../dataset/pssm/train_filtered_seqs.fasta -dbtype prot -out ../../dataset/pssm/train_db

## Calculate PSSM

### valid

In [10]:
!mkdir -p ../../dataset/pssm/valid_batch_pssms
!mkdir -p valid_tmp_seqs

In [ ]:
!awk '/^>/ {file="valid_tmp_seqs/"substr($1,2)".fasta"; print > file; next;} {print >> file;}' ../../dataset/ec40/valid.fasta

In [ ]:
!find valid_tmp_seqs -name "*.fasta" | xargs -P 24 -I {} sh -c 'base=$(basename {} .fasta); ncbi-blast-2.16.0+/bin/psiblast -query {} -db ../../dataset/pssm/valid_db -num_iterations 1 -out_ascii_pssm "../../dataset/pssm/valid_batch_pssms/$base.pssm" -save_pssm_after_last_round'

In [ ]:
!rm -rf valid_tmp_seqs

In [12]:
!grep '^>' ../../dataset/ec40/valid.fasta | wc -l

4921


In [11]:
!ls -1 ../../dataset/pssm/valid_batch_pssms | wc -l

4921


### test

In [ ]:
!mkdir -p ../../dataset/pssm/test_batch_pssms

!mkdir -p test_tmp_seqs

In [ ]:
!awk '/^>/ {file="test_tmp_seqs/"substr($1,2)".fasta"; print > file; next;} {print >> file;}' ../../dataset/ec40/test.fasta

In [ ]:
!find test_tmp_seqs -name "*.fasta" | xargs -P 24 -I {} sh -c 'base=$(basename {} .fasta); ncbi-blast-2.16.0+/bin/psiblast -query {} -db ../../dataset/pssm/test_db -num_iterations 1 -out_ascii_pssm "../../dataset/pssm/test_batch_pssms/$base.pssm" -save_pssm_after_last_round'

In [ ]:
!rm -rf test_tmp_seqs

In [13]:
!grep '^>' ../../dataset/ec40/test.fasta | wc -l

5459


In [14]:
!ls -1 ../../dataset/pssm/test_batch_pssms | wc -l

5459


### train

In [ ]:
!mkdir -p ../../dataset/pssm/train_batch_pssms
!mkdir -p train_tmp_seqs


In [ ]:
!awk '/^>/ {file="train_tmp_seqs/"substr($1,2)".fasta"; print > file; next;} {print >> file;}' ../../dataset/ec40/train.fasta

In [ ]:
!find train_tmp_seqs -name "*.fasta" | xargs -P 24 -I {} sh -c 'base=$(basename {} .fasta); ncbi-blast-2.16.0+/bin/psiblast -query {} -db ../../dataset/pssm/train_db -num_iterations 1 -out_ascii_pssm "../../dataset/pssm/train_batch_pssms/$base.pssm" -save_pssm_after_last_round'

In [3]:
!rm -rf train_tmp_seqs

In [1]:
!grep '^>' ../../dataset/ec40/train.fasta | wc -l

30967


In [2]:
!ls -1 ../../dataset/pssm/train_batch_pssms | wc -l

30967
